In [ ]:
!rm -rf /kaggle/working/Compression-Framework-for-EdgeAI
!git clone https://github.com/ha405/Compression-Framework-for-EdgeAI

Run the below command only once per session. If you reset session run again.

In [ ]:
# !pip install -r /kaggle/working/Compression-Framework-for-EdgeAI/requirements.txt
!pip install logbar
!pip install tokenicer
!pip install device_smi
!pip install random_word
!pip install datasets

In [2]:
import sys
import os
library_path = "/kaggle/working/Compression-Framework-for-EdgeAI/KLAWQ"
if library_path not in sys.path:
     sys.path.insert(0, library_path)
     print(f"Added '{library_path}' to sys.path")
from quant import GPTQModel, QuantizeConfig

Added '/kaggle/working/Compression-Framework-for-EdgeAI/KLAWQ' to sys.path

INFO  ENV: Auto setting PYTORCH_CUDA_ALLOC_CONF='expandable_segments:True' for memory saving.      
INFO  ENV: Auto setting CUDA_DEVICE_ORDER=PCI_BUS_ID for correctness.                              


2025-07-02 20:09:53.363981: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751486993.539829     248 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751486993.595094     248 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


## Imports

In [3]:
import os
# os.environ["HF_HOME"]           = os.path.abspath("/media/cvgl/5a7a01f8-7cab-46f8-a6d7-002c21d2008b1/haseeb/Compression-Framework-for-EdgeAI/hf_cache")
# os.environ["TRANSFORMERS_CACHE"]= os.path.abspath("/media/cvgl/5a7a01f8-7cab-46f8-a6d7-002c21d2008b1/haseeb/Compression-Framework-for-EdgeAI/hf_cache/models")
# os.environ["HF_DATASETS_CACHE"] = os.path.abspath("/media/cvgl/5a7a01f8-7cab-46f8-a6d7-002c21d2008b1/haseeb/Compression-Framework-for-EdgeAI/hf_cache/datasets")
import gc
import torch
import shutil
import math
import pandas as pd
from transformers import AutoTokenizer
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datasets import load_dataset, DatasetDict
from tqdm import tqdm
from typing import Dict, List
from torch.utils.data import Dataset

## WikiText-2

In [4]:
from datasets import load_dataset, DatasetDict

# Load only the 'test' and first 1000 'train' examples
dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split={
    "test": "test",
    "calibration": "train[:600]",
})

# Print the sizes of loaded splits
print({k: len(v) for k, v in dataset.items()})


{'test': 4358, 'calibration': 600}


To Clear GPU Cache

In [5]:
def clear_gpu_cache():
    gc.collect()
    torch.cuda.empty_cache()
    torch.cuda.ipc_collect()
    print("✅ GPU VRAM and cache cleared.")
clear_gpu_cache()

✅ GPU VRAM and cache cleared.


## Quantization Functions

In [6]:
class CalibDataset(Dataset):
    def __init__(self, tokenized: Dict[str, torch.Tensor]):
        self.input_ids      = tokenized["input_ids"]
        self.attention_mask = tokenized["attention_mask"]

    def __len__(self):
        return self.input_ids.size(0)

    def __getitem__(self, idx: int) -> Dict[str, torch.Tensor]:
        # Return torch.Tensors directly
        return {
            "input_ids":      self.input_ids[idx],
            "attention_mask": self.attention_mask[idx],
        }

def clear_quant_path(path=None):
    if path and os.path.exists(path):
        shutil.rmtree(path)
    torch.cuda.empty_cache(); gc.collect()

def clear_memory():
    torch.cuda.empty_cache(); gc.collect()

def quantize_and_eval(
    model_id: str,
    calib_tokenized: Dict[str, torch.Tensor],
    eval_texts: List[str],
    beta: float,
    tau: float,
    quant_path: str,
    tokenizer: AutoTokenizer,
    max_len: int,
    batch_size: int = 8
):
    print(f"\n→ [Quantize] β={beta}, τ={tau}")
    clear_quant_path(quant_path)

    # # 1) Load raw model
    quant_cfg = QuantizeConfig(bits=4, group_size=-1, beta=beta, tau=tau)
    model = GPTQModel.load(
        model_id,
        quantize_config=quant_cfg,
        trust_remote_code=True,
        torch_dtype="auto",
        device_map="auto"
    )

    # 2) Prepare a Dataset of Tensors (no Python lists!)
    calib_ds = CalibDataset(calib_tokenized)

    # 3) Quantize—GPTQ will now skip tensor-to-list conversions
    model.quantize(
        calibration_dataset=calib_ds,
        batch_size=batch_size,
        calibration_dataset_concat_size=len(calib_ds),
        calibration_enable_gpu_cache=False,
        buffered_fwd=False,
        auto_gc=True,
        tokenizer=tokenizer,    # ← Key to use the fast path
    )

    os.makedirs(os.path.dirname(quant_path), exist_ok=True)
    model.save(quant_path)
    print(f"   Quantization complete — saved to {quant_path}")
    del model
    clear_memory()

    # 4) Reload quantized model onto GPU(s)
    model = GPTQModel.from_pretrained(
        quant_path,
        trust_remote_code=True,
        device_map="auto",
        torch_dtype="auto",
        quantize_config=quant_cfg
    )
    model.eval()
    device = next(model.parameters()).device

    # 3) Build a single long list of token IDs (bypass HF max-length)
    all_ids = []
    for text in eval_texts:
        if not text.strip():
            continue
        all_ids += tokenizer.encode(text, add_special_tokens=False)

    # 4) Truncate to multiple of max_len and slice into blocks
    total = (len(all_ids) // max_len) * max_len
    if total == 0:
        raise ValueError(f"Need ≥{max_len} tokens, got {len(all_ids)}")
    ids_tensor = torch.tensor(all_ids[:total], dtype=torch.long, device=device)
    blocks = ids_tensor.view(-1, max_len)
    n_blocks = blocks.size(0)

    # 5) MagR-style block-wise PPL evaluation
    loss_fct = torch.nn.CrossEntropyLoss(reduction="none")
    total_nll, total_count = 0.0, 0

    for i in tqdm(range(0, n_blocks, batch_size), desc="Evaluating PPL"):
        batch = blocks[i : i + batch_size]         # [B, L]
        attn = torch.ones_like(batch, device=device)
        labels = batch.clone()
        labels[:, :-1] = batch[:, 1:]              # shift tokens
        labels[:, -1]  = -100                       # ignore last

        with torch.no_grad():
            out = model(input_ids=batch, attention_mask=attn, labels=labels)
            logits = out.logits[:, :-1, :].reshape(-1, out.logits.size(-1))
            labs   = labels[:, :-1].reshape(-1)
            nlls   = loss_fct(logits, labs) * (labs != -100).float()

        total_nll   += nlls.sum().item()
        total_count += (labs != -100).sum().item()

    avg_nll = total_nll / total_count
    ppl     = math.exp(avg_nll)
    print(f"     Eval complete: loss={avg_nll:.4f}, ppl={ppl:.2f}")

    del model
    clear_quant_path(quant_path)
    return avg_nll, ppl


In [ ]:
from huggingface_hub import login
from getpass import getpass

HF_TOKEN = getpass("Enter your Hugging Face token:")
login(token=HF_TOKEN)

print("Hugging Face login successful!")

Max len is for llama. Adjust Batch Size if Out of memory error. reduce to 2

In [7]:
calib_texts = dataset["calibration"]["text"]
eval_texts  = [t for t in dataset["test"]["text"] if t.strip()]

# --- init tokenizer & pre-tokenize calibration set ---
model_id = "meta-llama/Llama-2-7b-hf"
tokenizer = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
max_len = tokenizer.model_max_length
max_len = 4096
calib_tokenized = tokenizer(
    calib_texts,
    truncation=True,
    padding="longest",
    max_length=max_len,
    return_tensors="pt"
)

base_quant_path = "/kaggle/working/llama2-7b-quant"
beta_values     = [0.2]
tau_values      = []
results = []
total_iters = len(beta_values) + len(tau_values)
iter_count = 0


for beta in beta_values:
    iter_count += 1
    print(f"[Iter {iter_count}/{total_iters}] β={beta}, τ=0.5")
    qp = f"{base_quant_path}-b{beta}-t0.5"
    loss, ppl = quantize_and_eval(
        model_id=model_id,
        calib_tokenized=calib_tokenized,
        eval_texts=eval_texts,
        beta=beta,
        tau=1.0,
        quant_path=qp,
        tokenizer=tokenizer,
        max_len=max_len,
        batch_size=1
    )
    results.append({"beta": beta, "tau": 0.5, "loss": loss, "ppl": ppl})

# … select best_beta …

for tau in tau_values:
    iter_count += 1
    print(f"[Iter {iter_count}/{total_iters}] β={best_beta}, τ={tau}")
    qp = f"{base_quant_path}-b{best_beta}-t{tau}"
    loss, ppl = quantize_and_eval(
        model_id=model_id,
        calib_tokenized=calib_tokenized,
        eval_texts=eval_texts,
        beta=best_beta,
        tau=tau,
        quant_path=qp,
        tokenizer=tokenizer,      # ← and here too
        max_len=max_len,
        batch_size=1
    )
    results.append({"beta": best_beta, "tau": tau, "loss": loss, "ppl": ppl})

df = pd.DataFrame(results)
print(df.to_markdown(index=False))


[Iter 1/1] β=0.2, τ=0.5

→ [Quantize] β=0.2, τ=1.0
INFO  Estimated Quantization BPW (bits per weight): 4 bpw, based on [bits: 4, group_size: -1]      


Fetching 17 files:   0%|          | 0/17 [00:00<?, ?it/s]

INFO  Loader: Auto dtype (native float16): `torch.float16`                                         


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

INFO  Model: Loaded `generation_config`: GenerationConfig {
  "bos_token_id": 1,
  "do_sample": true,
  "eos_token_id": 2,
  "max_length": 4096,
  "pad_token_id": 0,
  "temperature": 0.6,
  "top_p": 0.9
}

INFO  Kernel: loaded -> `[]`                                                                       
INFO  Packing Kernel: selected: `TorchQuantLinear`                                                 
INFO  Hooked Modules: Using tree based config for accurate targeting of modules                    
INFO  Process: progress logs for `gptq` will be streamed to file: `gptq_log_avoided_time_07_02_2025_20h_10m_31s.log`
INFO  -----------------------------------------------------------------------------------------------------------------------------------------------------
INFO  | process     | layer     | module               | loss             | samples     | damp        | time      | fwd_time     | max_vram               |
INFO  -----------------------------------------------------------

Evaluating PPL: 100%|██████████| 81/81 [05:01<00:00,  3.72s/it]


     Eval complete: loss=1.7466, ppl=5.74
|   beta |   tau |   loss |     ppl |
|-------:|------:|-------:|--------:|
|    0.2 |   0.5 | 1.7466 | 5.73506 |


## Plotting

In [ ]:
df2 = pd.DataFrame(results2)

# Constants
const_tau = 0.5
df_beta = df[df['tau'] == const_tau].reset_index(drop=True)

best_beta = df_beta.loc[df_beta['ppl'].idxmin(), 'beta']
df_tau = df[df['beta'] == best_beta].reset_index(drop=True)

def plot_zoomed_bar(x, y, xlabel, ylabel, title, cmap):
    colors = cmap(np.linspace(0, 1, len(x)))
    fig, ax = plt.subplots(figsize=(8, 4))
    bars = ax.bar(x, y, color=colors, edgecolor='black', linewidth=0.8)

    ax.set_title(title, fontsize=14)
    ax.set_xlabel(xlabel, fontsize=12)
    ax.set_ylabel(ylabel, fontsize=12)
    ax.grid(axis='y', linestyle='--', alpha=0.6)
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)

    y_min, y_max = y.min(), y.max()
    margin = (y_max - y_min) * 0.15
    ax.set_ylim(y_min - margin, y_max + margin)

    plt.tight_layout()

plot_zoomed_bar(
    x=df_beta['beta'].astype(str),
    y=df_beta['ppl'],
    xlabel='β (Beta values)',
    ylabel='Perplexity',
    title='Perplexity vs Beta @ τ = 0.5',
    cmap=plt.cm.Set2
)

plot_zoomed_bar(
    x=df_beta['beta'].astype(str),
    y=df_beta['loss'],
    xlabel='β (Beta values)',
    ylabel='Avg NLL Loss',
    title='Loss vs Beta @ τ = 0.5',
    cmap=plt.cm.Pastel1
)

plot_zoomed_bar(
    x=df_tau['tau'].astype(str),
    y=df_tau['ppl'],
    xlabel='τ (Tau values)',
    ylabel='Perplexity',
    title=f'Perplexity vs Tau @ β = {best_beta}',
    cmap=plt.cm.Pastel2
)

plot_zoomed_bar(
    x=df_tau['tau'].astype(str),
    y=df_tau['loss'],
    xlabel='τ (Tau values)',
    ylabel='Avg NLL Loss',
    title=f'Loss vs Tau @ β = {best_beta}',
    cmap=plt.cm.Dark2
)

plt.show()

## For Debugging Only. If Model is saved in path

In [ ]:
def clear_memory():
    torch.cuda.empty_cache()
    gc.collect()
from torch.nn import CrossEntropyLoss
def load_quantized_model(quant_path: str, model_id: str, quant_cfg: QuantizeConfig):
    """Load the already‐quantized weights from disk."""
    return GPTQModel.from_pretrained(
        quant_path,
        trust_remote_code=True,
        device_map="auto",
        quantize_config=quant_cfg
    )

# ─── MagR-style block evaluation ────────────────────────────────────────────────────
def eval_magr(model, tokenizer, eval_texts, max_len, batch_size=8):
    device = next(model.parameters()).device
    # build flat token list
    all_ids = []
    for txt in eval_texts:
        if txt.strip():
            all_ids += tokenizer.encode(txt, add_special_tokens=False)
    # slice into non-overlapping blocks
    total = (len(all_ids) // max_len) * max_len
    ids = torch.tensor(all_ids[:total], device=device).view(-1, max_len)
    loss_fct = CrossEntropyLoss(reduction="none")
    nll = count = 0
    for i in tqdm(range(0, ids.size(0), batch_size), desc="MagR PPL"):
        batch = ids[i : i + batch_size]
        attn  = torch.ones_like(batch, device=device)
        labels = batch.clone()
        labels[:, :-1] = batch[:, 1:]
        labels[:, -1]  = -100
        with torch.no_grad():
            out    = model(input_ids=batch, attention_mask=attn, labels=labels)
            logits = out.logits[:, :-1, :].reshape(-1, out.logits.size(-1))
            labs   = labels[:, :-1].reshape(-1)
            nlls   = loss_fct(logits, labs) * (labs != -100).float()
        nll   += nlls.sum().item()
        count += (labs != -100).sum().item()
    avg_nll = nll / count
    return math.exp(avg_nll), avg_nll

# ─── Simple pad-masked evaluation ───────────────────────────────────────────────────
def eval_padmask(model, tokenizer, eval_texts, max_len, batch_size=8):
    device = next(model.parameters()).device
    # tokenize with padding
    enc = tokenizer(eval_texts,
                    return_tensors="pt",
                    padding="longest",
                    truncation=True,
                    max_length=max_len)
    input_ids = enc.input_ids.to(device)
    attn_mask = enc.attention_mask.to(device)
    loss_fct = CrossEntropyLoss(ignore_index=-100, reduction="sum")
    total_nll = 0.0
    total_tokens = 0
    for i in tqdm(range(0, input_ids.size(0), batch_size), desc="Pad-mask PPL"):
        b_ids  = input_ids[i : i + batch_size]
        b_mask = attn_mask[i : i + batch_size]
        labels = b_ids.clone()
        labels[b_mask == 0] = -100
        with torch.no_grad():
            out = model(input_ids=b_ids, attention_mask=b_mask, labels=labels)
        # out.loss is mean over non-ignored tokens
        num = (labels != -100).sum().item()
        total_nll += out.loss.item() * num
        total_tokens += num
    avg_nll = total_nll / total_tokens
    return math.exp(avg_nll), avg_nll

# ─── Main: load and run ─────────────────────────────────────────────────────────────
model_id   = "meta-llama/Llama-2-7b-hf"
quant_path = "/content/llama2-7b-quant-b0.0-t0.5"  # your 4-bit checkpoint dir
tokenizer  = AutoTokenizer.from_pretrained(model_id, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token
max_len    = tokenizer.model_max_length  # 1024 for GPT-2
batch_size = 8
clear_memory()

# calibration config just to pass through loader (not used during eval)
quant_cfg = QuantizeConfig(bits=4, group_size=-1, beta=0.0, tau=0.5)

# load eval texts once
from datasets import load_dataset
ds = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
eval_texts = [t for t in ds["text"] if t.strip()]

clear_memory()
model = load_quantized_model(quant_path, model_id, quant_cfg)

# MagR-style
magr_ppl, magr_nll = eval_magr(model, tokenizer, eval_texts, max_len, batch_size)
print(f"MagR-style → PPL: {magr_ppl:.2f}, avg NLL: {magr_nll:.4f}")

# Pad-mask style
pad_ppl, pad_nll = eval_padmask(model, tokenizer, eval_texts, max_len, batch_size)
print(f"Pad-mask → PPL: {pad_ppl:.2f}, avg NLL: {pad_nll:.4f}")

# cleanup
del model
clear_memory()